In [1]:
import numpy as np
#load data from file导入txt数据
def load_data(filename):
    dataset = []
    file = open(filename)
    for line in file.readlines():
        lineArr = line.strip().split('\t')
        m = len(lineArr)
        dataset.append(lineArr[0:m])
    return np.array(dataset,dtype=np.float64)
           
import matplotlib.pyplot as plt
def disp_dataset(x):
    print (x.shape)
    #可视化一下
    plt.scatter(x[:,0],x[:,1],marker='x',color = 'r')
    plt.xlabel('x0')
    plt.ylabel('x1')
    plt.show()


In [2]:
#计算距离
def computeDistance(A,B):
    return np.sqrt(np.sum(np.square(A-B)))

#我们一开始应该随机初始化centroids ,随机选择k个样本点作为质心,而不是书上的某个随机值
def randCentroids(x,k):
    m,n = x.shape # m=80个数据，n=2维
    centroids = np.zeros((k,n)) #产生k个 [0 0]的数据
    randIndex = np.random.choice(m,k) #从m个数据中选k个randindex
    centroids = x[randIndex]
    return centroids

#计算x的均值，改变centroids
def change_centroids(x,idx,K):
    m,n = x.shape
    centroids = np.zeros((K,n))
    for i in range(K):
        index = np.where(idx[:,0].ravel() == i)
        centroids[i] = np.mean(x[index],axis=0)
    return centroids

def plot_data(x,idx,K,cent,centroids,cnt):
    m=x.shape[0]
    tmp_idx=[]
    color_list=['r','g','y','b','m','orange']
    tmp_idx=idx[:,0].tolist() #要把numpyfloat64转换为list int
    for i in range(m):
        plt.scatter(x[i,0],x[i,1],marker='x',color=color_list[int(tmp_idx[i])])
    
   # plt.scatter(centroids[:,0],centroids[:,1],marker='o',color = 'black',linewidths=10)
    
    for k in range(K):
        plt.scatter(centroids[k,0],centroids[k,1],marker='o',color = color_list[k],linewidths=10)
        plt.plot(np.array(cent["%d"%(k)])[:,0],np.array(cent["%d"%(k)])[:,1],"b-o",color=color_list[k])
    plt.xlabel('X1')
    plt.ylabel('X2')
    plt.title("figure %d"%(cnt))
    plt.show()

        

#为数据集x找到最近的质心,直到质心不变为止   
def kmeans(x,K):
    #1.初始化质心
    centroids = randCentroids(x,K)
    m = x.shape[0]
    idx = np.zeros((x.shape[0],2)) #记录x对应的质心的下标 + 平方误差
    clusterchanged = True #记录分类变没变化
    cent = {} #用字典，来存储各个质心的变化
    cnt=0 #记录丢点的次数
    
    # 初始化质心字典
    for k in range(K):
        cent["%d"%(k)] = []
        cent["%d"%(k)].append(centroids[k,:].tolist())
        
    while clusterchanged:
        
        clusterchanged = False
        #2.为每个样本点分配质心
        for i in range(m):
            minDist = np.inf
            minIndex = -1
            for j in range(K):
                distance = computeDistance(x[i,:],centroids[j,:])
                if distance<minDist:
                    minDist = distance
                    minIndex = j
            if idx[i,0] != minIndex:
                clusterchanged = True
            idx[i,:] = minIndex, minDist**2
        #丢完点之后的情况输出一下
        cnt=cnt+1
        plot_data(x,idx,K,cent,centroids,cnt)
        #3.更新质心的位置
        centroids = change_centroids(x,idx,K)
        # 4.更新质心字典
        for k in range(K):
            cent["%d"%(k)].append(centroids[k,:].tolist())
    return centroids,idx,cent



In [3]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
def choose(filename,K):
    x = load_data(filename)
    disp_dataset(x)
    centroids,idx,cent = kmeans(x,K)

K_widget=widgets.IntSlider(value=4,min=1, max=6, step=1, continuous_update=False)
filename_widget=widgets.Dropdown(
    options=['testSet.txt','testSet2.txt'],
    description='选择数据集:',
    disabled=False,
)
interact(choose, filename=filename_widget,K=K_widget);


interactive(children=(Dropdown(description='选择数据集:', options=('testSet.txt', 'testSet2.txt'), value='testSet.t…